<a href="https://colab.research.google.com/github/aruaru0/Foursquare-Location-Matching/blob/main/IOU_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/datas/kaggle.json  ~/.kaggle/

In [2]:
!pip install lightgbm==3.3.1
!pip install onnxmltools
!pip install onnxruntime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 5.1 MB/s 
  Attempting uninstall: lightgbm
    Found existing installation: lightgbm 2.2.3
    Uninstalling lightgbm-2.2.3:
      Successfully uninstalled lightgbm-2.2.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 308 kB 5.1 MB/s 
     |████████████████████████████████| 278 kB 80.4 MB/s 
     |████████████████████████████████| 13.1 MB 82.0 MB/s 
     |████████████████████████████████| 78 kB 8.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.2 MB 4.7 MB/s 


In [3]:
!pip show lightgbm

Name: lightgbm
Version: 3.3.1
Summary: LightGBM Python Package
Home-page: https://github.com/microsoft/LightGBM
Author: None
Author-email: None
License: The MIT License (Microsoft)
Location: /usr/local/lib/python3.7/dist-packages
Requires: scikit-learn, scipy, wheel, numpy
Required-by: 


In [4]:
!pip install kaggle
!apt install unzip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-21ubuntu1.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [5]:
!chmod 600 /root/.kaggle/kaggle.json

In [6]:
!mkdir input output

In [7]:
!kaggle competitions download -c foursquare-location-matching

 90% 142M/159M [00:01<00:00, 134MB/s]
100% 159M/159M [00:01<00:00, 143MB/s]


In [8]:
!unzip foursquare-location-matching.zip -d input/foursquare-location-matching

Archive:  foursquare-location-matching.zip
  inflating: input/foursquare-location-matching/pairs.csv  
  inflating: input/foursquare-location-matching/sample_submission.csv  
  inflating: input/foursquare-location-matching/test.csv  
  inflating: input/foursquare-location-matching/train.csv  


In [9]:
#!kaggle kernels output aruaru0/training-data-for-binary-lgb-baseline-0-834 -p ./input/train_data

In [10]:
# !pip uninstall --yes lightgbm && pip install --install-option=--gpu lightgbm
!pip install optuna
!pip install pyarrow
!pip install fastparquet
!pip install catboost
!pip install ipywidgets
# !pip install eli5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 308 kB 4.9 MB/s 
     |████████████████████████████████| 209 kB 75.8 MB/s 
     |████████████████████████████████| 81 kB 10.8 MB/s 
     |████████████████████████████████| 78 kB 8.1 MB/s 
     |████████████████████████████████| 146 kB 83.0 MB/s 
     |████████████████████████████████| 112 kB 86.9 MB/s 
     |████████████████████████████████| 49 kB 7.1 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=5b329fb31378399f54798a91d0ea537303b9039588f3376e146019757e343f22
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |█████

## lightgbm

In [1]:
import lightgbm as lgb

import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.inspection import permutation_importance
import seaborn as sns
import pickle
import gc
import os
import pickle

import catboost as cb
from catboost import CatBoostClassifier
from catboost import Pool
from catboost import cv

import matplotlib.pyplot as plt

In [2]:
class CFG:
    seed = 46
    target = "point_of_interest"
    n_splits = 5

#    n_neighbors = 25
    is_debug = False

    model_path = '/content/drive/MyDrive/datas/foursquare/lightgbm'

In [3]:
def reduce_memory(df):
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type != object:
            cmin = df[col].min()
            cmax = df[col].max()
            if str(col_type)[:3] == 'int':
                if cmin > np.iinfo(np.int8).min and cmax < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif cmin > np.iinfo(np.int16).min and cmax < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif cmin > np.iinfo(np.int32).min and cmax < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif cmin > np.iinfo(np.int64).min and cmax < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if cmin > np.finfo(np.float16).min and cmax < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif cmin > np.finfo(np.float32).min and cmax < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    return df

In [4]:
from sklearn.model_selection import GroupKFold, KFold, StratifiedKFold
from tqdm import tqdm

def add_fold(df) :
  kf = StratifiedKFold(n_splits=CFG.n_splits, shuffle=True, random_state=CFG.seed)
  for i, (trn_idx, val_idx) in enumerate(kf.split(df, df["label"], df["label"])):
      df.loc[val_idx, "fold"] = i
  return df

In [5]:
sel = "0703"
n_data = 10

## LOAD DATA 1st 

In [6]:
df = []
for e in tqdm(range(n_data)):
  d = reduce_memory(pd.read_csv(f'/content/drive/MyDrive/datas/foursquare/traindata{sel}/train_data{e+1}.csv'))
  d = add_fold(d)
  df.append(d)


100%|██████████| 10/10 [06:27<00:00, 38.78s/it]


In [7]:
train = pd.concat(df)
train = train.reset_index(drop=True)

In [8]:
train.head()

,id,match_id,kneighbors,kneighbors_country,kneighbors_categories,categories_index,label,latdiff,londiff,manhattan,...,categories2_gesh,categories2_leven,categories2_jaro,categories2_lcs,categories2_fuz,categories2_len_diff,categories2_nleven,categories2_nlcsk,categories2_nlcs,fold
0,E_000001272c6c5d,E_000001272c6c5d,0.0,0.0,0.0,35.0,1,0.0,0.0,0.0,...,1.0,0.0,1.0,4.0,100.0,0.0,0.0,1.0,1.0,3.0
1,E_00009ab517afac,E_00009ab517afac,0.0,0.0,0.0,103.0,1,0.0,0.0,0.0,...,1.0,0.0,1.0,5.0,100.0,0.0,0.0,1.0,1.0,3.0
2,E_0000c362229d93,E_0000c362229d93,0.0,0.0,0.0,79.0,1,0.0,0.0,0.0,...,1.0,0.0,1.0,5.0,100.0,0.0,0.0,1.0,1.0,3.0
3,E_0000c362229d93,E_0000c362229d93,0.0,0.0,0.0,103.0,1,0.0,0.0,0.0,...,1.0,0.0,1.0,5.0,100.0,0.0,0.0,1.0,1.0,1.0
4,E_0000c566a81ea1,E_0000c566a81ea1,0.0,0.0,0.0,376.0,1,0.0,0.0,0.0,...,1.0,0.0,1.0,13.0,100.0,0.0,0.0,1.0,1.0,2.0


In [9]:
for e in df:
  del e
del df
gc.collect()

63

In [10]:
# train['kdist_diff'] = (train['kdist'] - train['kdist_country']) /\
#                             train['kdist_country']
# train['kneighbors_mean'] = train[['kneighbors', 'kneighbors_country']].mean(axis = 1)

In [11]:
# train = reduce_memory(train)
len(train)

24508312

In [12]:
gc.collect()

150

In [13]:
with open(f'/content/drive/MyDrive/datas/foursquare/pkl/pickled_traindata{sel}.pkl', 'wb') as f:
    pickle.dump(train, f)

## LOAD DATA

In [14]:
with open(f'/content/drive/MyDrive/datas/foursquare/pkl/pickled_traindata{sel}.pkl', 'rb') as f:
    train = pickle.load(f)

In [15]:
# x = train[train['label']==1]
# y = train[train['label']==0].sample(frac=0.5)
# len(x), len(y)

In [16]:
# train = pd.concat([x,y])
# del x, y
gc.collect()

200

In [17]:
train.sample(20)

,id,match_id,kneighbors,kneighbors_country,kneighbors_categories,categories_index,label,latdiff,londiff,manhattan,...,categories2_gesh,categories2_leven,categories2_jaro,categories2_lcs,categories2_fuz,categories2_len_diff,categories2_nleven,categories2_nlcsk,categories2_nlcs,fold
12518914,E_8de9b4bef83389,E_757c127900faf7,4.0,3.0,NaN,NaN,0,-0.001067,0.001616,0.002684,...,0.111084,11.0,0.351318,2.0,22.0,0.615234,0.846191,0.399902,0.153809,1.0
9926762,E_792931a48fdae9,E_669a9dc86c4a28,1.0,3.0,NaN,NaN,0,-0.013351,0.008354,0.021698,...,0.105286,12.0,0.347656,2.0,21.0,0.643066,0.856934,0.399902,0.142822,2.0
4207588,E_2e5360ff724f4e,E_11f22c4a57ee86,NaN,NaN,2.0,28.0,0,0.211426,-0.618652,0.830566,...,1.000000,0.0,1.000000,16.0,100.0,0.000000,0.000000,1.000000,1.000000,4.0
2066204,E_11fbef83802ece,E_4279918fb9ec49,NaN,NaN,1.0,314.0,0,-0.221924,3.142578,3.363281,...,0.428467,24.0,0.854492,9.0,43.0,0.727051,0.727051,1.000000,0.272705,3.0
2267755,E_0527303e62b68c,E_7c7ac4c9143e2e,NaN,NaN,NaN,NaN,0,-1.634766,8.937500,10.570312,...,1.000000,0.0,1.000000,12.0,100.0,0.000000,0.000000,1.000000,1.000000,0.0
20795428,E_da8854e17c86b5,E_74c8cd373542af,20.0,NaN,NaN,NaN,0,-0.000240,0.000418,0.000657,...,NaN,NaN,NaN,NaN,NaN,0.799805,NaN,NaN,NaN,1.0
21319055,E_df70ea7258819d,E_7f14353e2fd74a,29.0,NaN,NaN,NaN,0,-0.115479,0.050140,0.165649,...,0.428467,11.0,0.638672,6.0,43.0,0.333252,0.687500,0.375000,0.500000,2.0
19597217,E_cb71be92d9e46e,E_05322341046fbc,NaN,NaN,NaN,NaN,0,19.718750,-12.804688,32.531250,...,1.000000,0.0,1.000000,12.0,100.0,0.000000,0.000000,1.000000,1.000000,3.0
21600126,E_cef1df34464d26,E_6fa52872240f53,NaN,NaN,8.0,232.0,0,3.976562,1.850586,5.828125,...,0.790527,9.0,0.727539,17.0,79.0,0.529297,0.346191,0.653809,1.000000,2.0
23599540,E_fe11419f905785,E_144b2ad07d2e30,26.0,NaN,NaN,NaN,0,0.001966,-0.000112,0.002079,...,0.394775,32.0,0.630371,17.0,45.0,0.451660,0.710938,0.377686,0.548340,1.0


In [18]:
train[['categories_lcs', 'categories2_lcs']].sample(20)

,categories_lcs,categories2_lcs
12185831,12.0,12.0
12598130,3.0,3.0
21913258,12.0,12.0
978114,NaN,NaN
7191752,12.0,12.0
23098841,8.0,6.0
21698638,7.0,7.0
4034815,3.0,3.0
21788176,19.0,19.0
1461932,7.0,7.0


In [19]:
len(train)

24508312

## START

In [20]:
# str | Noneって書きたいところだけど3.7系でもいけるようにOptionalを使う
from typing import Optional
from collections import OrderedDict
from lightgbm.callback import CallbackEnv
from tqdm.auto import tqdm

class LgbmProgressBarCallback:
    description: Optional[str]
    pbar: tqdm

    def __init__(self, description: Optional[str] = None):
        self.description = description
        self.pbar = tqdm()

    def __call__(self, env: CallbackEnv):

        # 初回だけProgressBarを初期化する
        is_first_iteration: bool = env.iteration == env.begin_iteration

        if is_first_iteration:
            total: int = env.end_iteration - env.begin_iteration
            self.pbar.reset(total=total)
            self.pbar.set_description(self.description, refresh=False)

        # valid_setsの評価結果を更新
        if len(env.evaluation_result_list) > 0:
            # OrderedDictにしないと表示順がバラバラになって若干見にくい
            postfix = OrderedDict(
                [
                    (f"{entry[0]}:{entry[1]}", str(entry[2]))
                    for entry in env.evaluation_result_list
                ]
            )
            self.pbar.set_postfix(ordered_dict=postfix, refresh=False)

        # 進捗を1進める
        self.pbar.update(1)
        self.pbar.refresh()


In [21]:
import optuna.integration.lightgbm as lgbopt

In [22]:
def sample_scheduler_func(current_lr, eval_history, best_round, is_higher_better):
    """次のラウンドで用いる学習率を決定するための関数 (この中身を好きに改造する)

    :param current_lr: 現在の学習率 (指定されていない場合の初期値は None)
    :param eval_history: 検証用データに対する評価指標の履歴
    :param best_round: 現状で最も評価指標の良かったラウンド数
    :param is_higher_better: 高い方が性能指標として優れているか否か
    :return: 次のラウンドで用いる学習率

    NOTE: 学習を打ち切りたいときには callback.EarlyStopException を上げる
    """
    # 学習率が設定されていない場合のデフォルト
    #current_lr = current_lr or 0.05


    # 試しに 20 ラウンド毎に学習率を半分にしてみる
    # if len(eval_history) % 20 == 0:
    #     current_lr /= 2

    # # 小さすぎるとほとんど学習が進まないので下限も用意する
    # min_threshold = 0.001
    # current_lr = max(min_threshold, current_lr)
    n = len(eval_history) 
    
    if n == 200 : # 1.0 -> 0.5
      current_lr *= 5
    elif n == 5000 : # 0.5 -> 0.2 
      current_lr /= 5
    
    return current_lr


class LrSchedulingCallback(object):
    """ラウンドごとの学習率を動的に制御するためのコールバック"""

    def __init__(self, strategy_func):
        # 学習率を決定するための関数
        self.scheduler_func = strategy_func
        # 検証用データに対する評価指標の履歴
        self.eval_metric_history = []

    def __call__(self, env):
        # 現在の学習率を取得する
        current_lr = env.params.get('learning_rate')

        # 検証用データに対する評価結果を取り出す (先頭の評価指標)
        first_eval_result = env.evaluation_result_list[0]
        # スコア
        metric_score = first_eval_result[2]
        # 評価指標は大きい方が優れているか否か
        is_higher_better = first_eval_result[3]

        # 評価指標の履歴を更新する
        self.eval_metric_history.append(metric_score)
        # 現状で最も優れたラウンド数を計算する
        best_round_find_func = np.argmax if is_higher_better else np.argmin
        best_round = best_round_find_func(self.eval_metric_history)

        # 新しい学習率を計算する
        new_lr = self.scheduler_func(current_lr=current_lr,
                                     eval_history=self.eval_metric_history,
                                     best_round=best_round,
                                     is_higher_better=is_higher_better)

        # 次のラウンドで使う学習率を更新する
        update_params = {
            'learning_rate': new_lr,
        }
        env.model.reset_parameter(update_params)
        env.params.update(update_params)

    @property
    def before_iteration(self):
        # コールバックは各イテレーションの後に実行する
        return False

In [23]:
def fit_lgbm_optuna(X, y, params=None, es_rounds=20, seed=42, N_SPLITS=5, 
             n_class=None, model_dir=None, folds=None):
  models = []
  oof = []
  
  for i in range(CFG.n_splits):
    print(f"== fold {i} ==")
    trn_idx = folds!=i
    val_idx = folds==i
    # train, validデータに分割
    X_train, y_train = X[trn_idx], y.iloc[trn_idx]
    X_valid, y_valid = X.iloc[val_idx], y.iloc[val_idx]


    train_data = lgb.Dataset(
        data=X_train, 
        label=y_train, 
        # weight=compute_sample_weight(class_weight='balanced', y=y_train).astype('float32'),
    )
    validation_data = lgb.Dataset(
        data=X_valid, 
        label=y_valid, 
        # weight=np.ones(len(X_valid)).astype('float32'),
    )

    model = lgbopt.train(
      params=params,
      train_set=train_data, 
      num_boost_round=500,
      early_stopping_rounds=50,
      valid_sets=[validation_data],
      verbose_eval=False,
      callbacks=[LgbmProgressBarCallback(description="Model A"),
                #  LrSchedulingCallback(strategy_func=sample_scheduler_func),
                ],
    )
    print("best_params " , model.params)
    # model.save_model(f'{CFG.model_path}/lgbm_fold{i}.txt', num_iteration=model.best_iteration)


    # validデータに対して予測し、結果を格納 
    pred = model.predict(X_valid, num_iteration=model.best_iteration)
    pred = pred.round(0)
    accuracy = accuracy_score(pred, y_valid) 
    print()
    print(accuracy)
    oof.append(pred)
    models.append(model)
    break

  return oof, models


### lightgbm

In [24]:
def fit_lgbm2(X, y, params=None, es_rounds=20, seed=42, N_SPLITS=5, 
             n_class=None, model_dir=None, folds=None):
  models = []
  oof = []
  
  for i in range(CFG.n_splits):
    print(f"== fold {i} ==")
    trn_idx = folds!=i
    val_idx = folds==i
    # train, validデータに分割
    X_train, y_train = X[trn_idx], y.iloc[trn_idx]
    X_valid, y_valid = X.iloc[val_idx], y.iloc[val_idx]


    train_data = lgb.Dataset(
        data=X_train, 
        label=y_train, 
        # weight=compute_sample_weight(class_weight='balanced', y=y_train).astype('float32'),
    )
    validation_data = lgb.Dataset(
        data=X_valid, 
        label=y_valid, 
        # weight=np.ones(len(X_valid)).astype('float32'),
    )

    model = lgb.train(
      params=params,
      train_set=train_data, 
      # num_boost_round=100000, 
      # early_stopping_rounds=50, # 20->50
      valid_sets=[validation_data],
      verbose_eval=False,
      callbacks=[LgbmProgressBarCallback(description="Model A"),
                 LrSchedulingCallback(strategy_func=sample_scheduler_func),
                ],
    )
    # print("best_params " , model.params)
    # model.save_model(f'{CFG.model_path}/lgbm_cat2_fold{i}.txt', num_iteration=model.best_iteration)
    model.save_model(f'{CFG.model_path}/lgbm{sel}_fold{i}.txt', num_iteration=model.best_iteration)

    # validデータに対して予測し、結果を格納 
    pred = model.predict(X_valid, num_iteration=model.best_iteration)
    pred = pred.round(0)
    accuracy = accuracy_score(pred, y_valid) 
    print()
    print(accuracy)
    # oof.append(pred)
    models.append(model)

  return models


### catboost

In [25]:
def fit_catboost(X, y, params=None, es_rounds=20, seed=42, N_SPLITS=5, 
                 n_class=None, model_dir=None, folds=None):
  
  for i in range(CFG.n_splits):
    print(f"== fold {i} ==")
    trn_idx = folds!=i
    val_idx = folds==i
    # train, validデータに分割
    X_train, y_train = X[trn_idx], y.iloc[trn_idx]
    X_valid, y_valid = X.iloc[val_idx], y.iloc[val_idx]

    # categorical_features_names = ['kneighbors_categories']
    train_data = Pool(
        data=X_train, 
        label=y_train, 
        # cat_features=categorical_features_names
    )
    validation_data = Pool(
        data=X_valid, 
        label=y_valid, 
        # cat_features=categorical_features_names
    )


    # パラメータを指定した場合は、以下のようにインスタンスに適用させる
    model = CatBoostClassifier(**params)


    model.fit(train_data, 
          eval_set=validation_data,    # 検証用データ
          # early_stopping_rounds=20,  # 10回以上精度が改善しなければ中止
          use_best_model=True,       # 最も精度が高かったモデルを使用するかの設定
          verbose=100,
          plot=False)                 # 誤差の推移を描画するか否かの設定

    # cv_params = model.get_params()
    # cv_data = cv(pool=Pool(X_valid, y_valid), 
    #          params=cv_params, 
    #          plot=True, 
    #          nfold=10, 
    #          early_stopping_rounds=10
    # )

    # model = cat.train(
    #   params=params,
    #   train_set=train_data, 
    #   num_boost_round=100000, 
    #   early_stopping_rounds=20,
    #   valid_sets=[validation_data],
    #   verbose_eval=False,
    #   callbacks=[LgbmProgressBarCallback(description="Model A"),
    #              #LrSchedulingCallback(strategy_func=sample_scheduler_func),
    #             ],
    # )
    # # print("best_params " , model.params)



    model.save_model(f'{CFG.model_path}/cat{sel}_fold{i}.cbm')#, num_iteration=model.best_iteration)
    # validデータに対して予測し、結果を格納 
    pred = model.predict(X_valid)
    pred = pred.round(0)
    accuracy = accuracy_score(pred, y_valid) 
    print()
    print(accuracy)
    del train_data,validation_data,X_train, y_train,X_valid, y_valid
    gc.collect()
  return model


In [26]:
# train['kneighbors_categories'] = train['kneighbors_categories'].astype('category')

In [27]:
# type(train['kneighbors_categories'])

### features

In [28]:
# features = ['kdist', 'kneighbors', 'kneighbors_country',
#        'kneighbors_categories', 'categories_index', 'latdiff',
#        'londiff', 'manhattan', 'euclidean', 'haversine', 'name_sim',
#        'name_gesh', 'name_leven', 'name_jaro', 'name_lcs', 'name_fuz',
#        'name_len_diff', 'name_nleven', 'name_nlcsk', 'name_nlcs',
#        'address_sim', 'address_gesh', 'address_leven', 'address_jaro',
#        'address_lcs', 'address_fuz', 'address_len_diff', 'address_nleven',
#        'address_nlcsk', 'address_nlcs', 'city_gesh', 'city_leven', 'city_jaro',
#        'city_lcs', 'city_fuz', 'city_len_diff', 'city_nleven', 'city_nlcsk',
#        'city_nlcs', 'state_sim', 'state_gesh', 'state_leven', 'state_jaro',
#        'state_lcs', 'state_fuz', 'state_len_diff', 'state_nleven',
#        'state_nlcsk', 'state_nlcs', 'zip_gesh', 'zip_leven', 'zip_jaro',
#        'zip_lcs', 'zip_fuz', 'url_sim', 'url_gesh', 'url_leven', 'url_jaro',
#        'url_lcs', 'url_fuz', 'url_len_diff', 'url_nleven', 'url_nlcsk',
#        'url_nlcs', 'phone_gesh', 'phone_leven', 'phone_jaro', 'phone_lcs',
#        'phone_fuz', 'categories_sim', 'categories_gesh', 'categories_leven',
#        'categories_jaro', 'categories_lcs', 'categories_fuz',
#        'categories_len_diff', 'categories_nleven', 'categories_nlcsk',
#        'categories_nlcs', 'country_sim', 'country_gesh', 'country_leven',
#        'country_jaro', 'country_lcs', 'country_fuz', 'country_len_diff',
#        'country_nleven', 'country_nlcsk', 'country_nlcs']

In [29]:
# features = ['kneighbors', 'kneighbors_country',
#        'kneighbors_categories', 'categories_index', 'latdiff',
#        'londiff', 'manhattan', 'euclidean', 'haversine', 'name_sim',
#        'name_gesh', 'name_leven', 'name_jaro', 'name_lcs', 'name_fuz',
#        'name_len_diff', 'name_nleven', 'name_nlcsk', 'name_nlcs',
#        'address_sim', 'address_gesh', 'address_leven', 'address_jaro',
#        'address_lcs', 'address_fuz', 'address_len_diff', 'address_nleven',
#        'address_nlcsk', 'address_nlcs', 'city_gesh', 'city_leven', 'city_jaro',
#        'city_lcs', 'city_fuz', 'city_len_diff', 'city_nleven', 'city_nlcsk',
#        'city_nlcs', 'state_sim', 'state_gesh', 'state_leven', 'state_jaro',
#        'state_lcs', 'state_fuz', 'state_len_diff', 'state_nleven',
#        'zip_gesh', 'zip_jaro',
#        'zip_lcs', 'url_sim',
#        'url_lcs', 'phone_gesh', 'phone_jaro', 'phone_lcs',
#        'categories_sim', 'categories_gesh', 'categories_leven',
#        'categories_jaro', 'categories_lcs', 'categories_fuz',
#        'categories_len_diff', 'categories_nleven', 'categories_nlcsk',
#        'categories_nlcs',
#        'categories2_sim',
#        'categories2_gesh', 'categories2_leven', 'categories2_jaro',
#        'categories2_lcs', 'categories2_fuz', 'categories2_len_diff',
#        'categories2_nleven', 'categories2_nlcsk', 'categories2_nlcs']

In [30]:
features = [
      # 'kneighbors', 'kneighbors_country','kneighbors_categories', 
       'categories_index', 
       'latdiff',
       'londiff', 'manhattan', 'euclidean', 'haversine', 'name_sim',
       'name_gesh', 'name_leven', 'name_jaro', 'name_lcs', 'name_fuz',
       'name_len_diff', 'name_nleven', 'name_nlcsk', 'name_nlcs',
       'address_sim', 'address_gesh', 'address_leven', 'address_jaro',
       'address_lcs', 'address_fuz', 'address_len_diff', 'address_nleven',
       'address_nlcsk', 'address_nlcs', 'city_gesh', 'city_leven', 'city_jaro',
       'city_lcs', 'city_fuz', 'city_len_diff', 'city_nleven', 'city_nlcsk',
       'city_nlcs', 
       'phone_gesh', 'phone_jaro', 'phone_lcs',
       'categories_sim', 'categories_gesh', 'categories_leven',
       'categories_jaro', 'categories_lcs', 'categories_fuz',
       'categories_len_diff', 'categories_nleven', 'categories_nlcsk',
       'categories_nlcs',       
       'categories',
       'categories2_sim',
       'categories2_gesh', 'categories2_leven', 'categories2_jaro',
       'categories2_lcs', 'categories2_fuz', 'categories2_len_diff',
       'categories2_nleven', 'categories2_nlcsk', 'categories2_nlcs']

In [31]:
 for e in train.columns : 
  if e not in features : print(e)

id
match_id
kneighbors
kneighbors_country
kneighbors_categories
label
phone_leven
phone_fuz
fold


In [32]:
for e in features :
  if e not in train.columns :
    print(e)

In [33]:
len(features), len(train.columns), train.columns, 

(59, 68, Index(['id', 'match_id', 'kneighbors', 'kneighbors_country',
        'kneighbors_categories', 'categories_index', 'label', 'latdiff',
        'londiff', 'manhattan', 'euclidean', 'haversine', 'name_sim',
        'name_gesh', 'name_leven', 'name_jaro', 'name_lcs', 'name_fuz',
        'name_len_diff', 'name_nleven', 'name_nlcsk', 'name_nlcs',
        'address_sim', 'address_gesh', 'address_leven', 'address_jaro',
        'address_lcs', 'address_fuz', 'address_len_diff', 'address_nleven',
        'address_nlcsk', 'address_nlcs', 'city_gesh', 'city_leven', 'city_jaro',
        'city_lcs', 'city_fuz', 'city_len_diff', 'city_nleven', 'city_nlcsk',
        'city_nlcs', 'phone_gesh', 'phone_leven', 'phone_jaro', 'phone_lcs',
        'phone_fuz', 'categories_sim', 'categories_gesh', 'categories_leven',
        'categories_jaro', 'categories_lcs', 'categories_fuz', 'categories',
        'categories_len_diff', 'categories_nleven', 'categories_nlcsk',
        'categories_nlcs', 'categorie

In [34]:
import math
weight = math.sqrt(sum(train['label']==0)/sum(train['label'] == 1))
# weight = math.sqrt(7)
weight

4.36963818653574

In [35]:
# train[[ 'name_sim',
#        'name_gesh', 'name_leven', 'name_jaro', 'name_lcs', 'name_fuz',
#        'name_len_diff', 'name_nleven', 'name_nlcsk', 'name_nlcs']].corr()

### fit

In [36]:
# train[features] = train[features].fillna(-1)
train

,id,match_id,kneighbors,kneighbors_country,kneighbors_categories,categories_index,label,latdiff,londiff,manhattan,...,categories2_gesh,categories2_leven,categories2_jaro,categories2_lcs,categories2_fuz,categories2_len_diff,categories2_nleven,categories2_nlcsk,categories2_nlcs,fold
0,E_000001272c6c5d,E_000001272c6c5d,0.0,0.0,0.0,35.0,1,0.000000,0.000000,0.000000,...,1.0,0.0,1.0,4.0,100.0,0.0,0.0,1.0,1.0,3.0
1,E_00009ab517afac,E_00009ab517afac,0.0,0.0,0.0,103.0,1,0.000000,0.000000,0.000000,...,1.0,0.0,1.0,5.0,100.0,0.0,0.0,1.0,1.0,3.0
2,E_0000c362229d93,E_0000c362229d93,0.0,0.0,0.0,79.0,1,0.000000,0.000000,0.000000,...,1.0,0.0,1.0,5.0,100.0,0.0,0.0,1.0,1.0,3.0
3,E_0000c362229d93,E_0000c362229d93,0.0,0.0,0.0,103.0,1,0.000000,0.000000,0.000000,...,1.0,0.0,1.0,5.0,100.0,0.0,0.0,1.0,1.0,1.0
4,E_0000c566a81ea1,E_0000c566a81ea1,0.0,0.0,0.0,376.0,1,0.000000,0.000000,0.000000,...,1.0,0.0,1.0,13.0,100.0,0.0,0.0,1.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24508307,E_fffa63a008cbcd,E_623477839dbabb,NaN,NaN,NaN,NaN,1,0.312744,1.310547,1.623047,...,1.0,0.0,1.0,21.0,100.0,0.0,0.0,1.0,1.0,3.0
24508308,E_fffe4d9910e5e4,E_08ef93e39cd689,NaN,NaN,NaN,NaN,0,-1.928711,1.716797,3.646484,...,1.0,0.0,1.0,39.0,100.0,0.0,0.0,1.0,1.0,0.0
24508309,E_fffe4d9910e5e4,E_1bd63fca6b958f,NaN,NaN,NaN,NaN,0,-1.794922,3.392578,5.187500,...,1.0,0.0,1.0,39.0,100.0,0.0,0.0,1.0,1.0,2.0
24508310,E_fffe4d9910e5e4,E_d8daa9ff89a124,NaN,NaN,NaN,NaN,0,-1.997070,1.594727,3.591797,...,1.0,0.0,1.0,39.0,100.0,0.0,0.0,1.0,1.0,3.0


In [37]:
is_gbm = True

In [38]:
if is_gbm :
  # learning_rate 0.05 -> 0.1
  params = {'task': 'train', 'boosting_type': 'gbdt', 
            'objective': 'binary', 'metric': 'binary_logloss', 
            'learning_rate': 0.05,
            'num_leaves': 256, 'max_depth': -1, 
            'min_data_in_leaf': 20, 
            'random_state': 42, 
            'lambda_l1': 0.7964782150381488,
            'max_bin': 255, 'feature_pre_filter': False, 
            'feature_fraction': 0.4,
            'bagging_fraction': 0.9277418926611961, 'bagging_freq': 5,
            'verbose': -1, 'lambda_l2': 0.7677209353759652, 
            'num_boost_round' : 100000,
            'early_stopping_rounds' : 100,
            # 'boosting_type' : 'dart'
            # 'scale_pos_weight' : weight,
            #'min_child_samples': 10,
            #'is_unbalance': True,
            }

  # params = {'task': 'train', 'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'binary_logloss',
  #           'random_state': 42, 
  #           'boosting_type' : 'dart'
  #           }

  # params['learning_rate'] = 0.1
  params['learning_rate'] = 0.075
  models = fit_lgbm2(train[features], train["label"].astype(int), 
                    params=params, 
                    n_class=int(train["label"].max() + 1), 
                    N_SPLITS=CFG.n_splits, 
                    folds=train["fold"].values)
else :
  # params = {
  #   'depth' :10,                 # 木の深さ
  #   'learning_rate' : 0.005,       # 学習率
  #   'early_stopping_rounds' : 50,
  #   'iterations' : 100000, 
  #   'loss_function': 'Logloss',
  #   'eval_metric' : 'Accuracy', # AUC or QueryAUC
  #   'random_seed' :42,
  #   'task_type': 'GPU',
  #   # 'boosting_type' : 'Ordered',
  #   'border_count' : 254,
  #   'l2_leaf_reg' : 2, #2-30
  #   # 'bagging_temperature' : 20 # 0-1
  #   # 'random_strength' : 1-20
  # }

  params = {
      'task_type':'GPU',
      #'depth' : 7,                  # 木の深さ
      'learning_rate' : 0.098,       # 学習率
      #'objective': 'Logloss',
      #'objective':['Recall', 'F1', 'Logloss'],
      'objective':'Logloss',
      # 'custom_metric': ['Recall:use_weights=False', 'Precision:use_weights=False', 'F1:use_weights=False'],
      #'eval_metric':'Recall:use_weights=False',
      'eval_metric': 'Recall', #F1:use_weights=False',
      # 'class_weights': [0.4, 0.6],
      #'scale_pos_weight':2.0,
      #'colsample_bylevel': 0.04423452661171374,
      'depth': 8,
      #'boosting_type': 'Plain',
      'bootstrap_type': 'Bayesian',
      'iterations': 40000,
      #'learning_rate': 0.19879419205122947,
      'learning_rate': 0.08879419205122947,
      #'learning_rate': 0.08879419205122947,
      'early_stopping_rounds' : 5000,        
      'bagging_temperature': 1.4706304318051489,        
      #'custom_loss' :['AUC'], 
      #'custom_loss':['Logloss'],
      'random_seed' :42,
      'verbose': 250  
  }

  # params['learning_rate'] = 0.01
  print("catboost:")
  fit_catboost(train[features], train["label"].astype(int), 
                params=params, 
                n_class=int(train["label"].max() + 1), 
                N_SPLITS=CFG.n_splits, 
                folds=train["fold"].values)

== fold 0 ==


0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "



0.9936507275896512
== fold 1 ==


0it [00:00, ?it/s]

...

/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "



0.9935660598384792
== fold 4 ==


0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "



0.9935585087500969


In [ ]:
# 0.025155366263405532　0.024608290268635095

In [ ]:
# import onnxmltools
# from onnxconverter_common import *

In [ ]:
# models

In [ ]:
# features_count = len(features)

# for i, model in enumerate(models):
#   onnx_model = onnxmltools.convert_lightgbm(model, name='LightGBM', initial_types=[['input', FloatTensorType([0, features_count])]])
#   onnxmltools.utils.save_model(onnx_model, 'model.onnx')
#   m = CatBoostClassifier()
#   m.load_model('model.onnx', format='onnx')
#   m.save_model(f'/content/drive/MyDrive/datas/foursquare/catboost/cat{sel}_fold{i}.cbm')#, num_iteration=model.best_iteration)

In [ ]:
# 0.9907111576784912 org
# 0.9899762201446902 lr=0.1
# 0.989910582018156 lr=0.1 2000
# 0.9898274869430753 lr=0.1 2000 lr schedule off
# kr 0.05 2000 0.990449652589267
# kr 0.055 2000 0.9905097044922664
# kr 0.06 2000 0.9905502046128939
# kr 0.07 2000 0.9906004806247074
# kr 0.08 2000 0.9902715917140943


In [ ]:
#weight 1.414 0.9908714124661466
#weight 2.236 0.991146533975237
#weight 2.6457 0.9911873832348355
#weight 3.5740 0.991203443627498

In [ ]:
  # catboost
# 0.9842255521545541 bestIteration = 2906
# 0.9849067222868323
# 0.9845785316541611 bestIteration = 3086
# 0.9843173757
# 0.9891393188638303

In [ ]:
 # 0.988324015262026 use all
# 0.9885390310873633 0.03455205829664009 remove country...
# 0.9885672829159683 0.03433813117916073
# 0.9891246648787949 0.03248566124792865 add categories2
# 0.9891780831674574 0.032543843951041096 'state_nlcsk', 'state_nlcs'
# 

In [ ]:
# from sklearn.metrics import make_scorer
# def acc(y_true, y_pred):
#     y_pred = y_pred.round(0)
#     return accuracy_score(y_true, y_pred)
# acc_scorer = make_scorer(acc)

# X_train, y_train = train[features], train["label"].astype(int)
# result = permutation_importance(model, X_train, y_train, scoring=acc_scorer, n_repeats=10, n_jobs=-1, random_state=71)
# perm_imp_df = pd.DataFrame({"importances_mean":result["importances_mean"], "importances_std":result["importances_std"]}, index=X.columns)
# print("[Permutation feature Importance]")
# display(perm_imp_df)

In [ ]:
#0.9881529946506996


In [ ]:
# 0.9886262818271736
# 0.9885151008269207 0.033652739798557055
# 0.9887625196165962 0.03250554427692221  TIMEUP!! 9hour 255MB
# 0.9885361503332578 0.033437097701619414
#----
# 0.9889233422215168 0.0326120642504442
# 0.9893850763058167 0.030717233208053953 
# 0.9897223596475375 0.030133636490857006 weighted　

In [ ]:
#0.9832166157227484
#0.986433063288391
#0.9879528738117864 0.9877047494263559
#0.9904222504359598 0.027791652491275087